## Exercise 04 : A/B-testing

Import libraries

In [1]:
import pandas as pd
import sqlite3

database_file = "../data/checking-logs.sqlite"

* create a connection to the database using the library sqlite3

In [2]:
connection_obj = sqlite3.connect(database_file)
cursor_obj = connection_obj.cursor()

  * using only one query for each of the groups, create two dataframes: test_results
  and control_results with the columns time and avg_diff and only two rows
    * time should have the values: after and before
    * avg_diff contains the average delta among all the users for the time period
    before each of them made their first visit to the page and afterward
    * only take into account the users that have observations before and after
  * we still are not using the lab ’project1’

In [3]:
def get_results_df(table_name: str, con: object) -> pd.DataFrame:
    query = f"""
        WITH user_diffs AS (
            SELECT 
                uid,
                first_commit_ts,
                first_view_ts,
                d.deadlines,
                (strftime('%s', first_commit_ts) - d.deadlines) / 3600 AS diff, -- convert first_commit to unix sec, then convert delta to hours
                CASE 
                    WHEN first_commit_ts < first_view_ts THEN 'before'
                    ELSE 'after'
                END AS time
            FROM {table_name}
            JOIN deadlines d 
                ON labname = d.labs
            WHERE labname <> 'project1'
        ),
        valid_users AS (
            SELECT uid FROM user_diffs 
            GROUP BY uid 
            HAVING COUNT(DISTINCT time) = 2 -- account users only with "before" and "after"
        )
        SELECT 
            time, 
            AVG(diff) AS avg_diff
        FROM user_diffs
        WHERE uid in (SELECT uid FROM valid_users)
        GROUP BY time;
    """

    return pd.read_sql(query, con)


test_results = get_results_df(table_name="test", con=connection_obj)
control_results = get_results_df(table_name="control", con=connection_obj)


print("test_results dataframe:")
display(test_results)
print("control_results dataframe:")
display(control_results)

test_results dataframe:


,time,avg_diff
0,after,-104.6000
1,before,-60.5625


control_results dataframe:


,time,avg_diff
0,after,-117.636364
1,before,-99.464286


* close the connection

In [4]:
connection_obj.close()

### Вывод 

Гипотеза верна, так как в тестовой группе после посещения Newsfeed среднее время (дельта между комитом и дедлайном) уменьшилось с -60 до -104. Это говорит о том, что после просмотра Newsfeed студенты в среднем начали работать над лабораторнами раньше.

В контрольной группе, где посещений не было (в значении первого захода на Newsfeed стоит avg дата), так же наблюдается уменьшее дельты, однако оно значительно меньше, по сравнению с тестовой группой. Это говорит о естественной тенденции студентов начинать работу немного раньше, но эффект в тестовой группе значительно сильнее.

Таким образом, можно сделать вывод, что Newsfeed действительно оказал влияние на поведение студентов, побудив их начинать работу над лабораторными раньше. Это подтверждает гипотезу, и внедрение страницы Newsfeed для всей группы кажется оправданным.